In [1]:
!pip install fsspec

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/fsspec-2023.6.0.dist-info/METADATA'



In [2]:
!pip install pytz

In [3]:
!pip install s3fs

  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.6.0-py3-none-any.whl (176 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.


In [4]:
!pip install dask[complete]

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/fsspec-2023.6.0.dist-info/METADATA'



In [5]:
# Specify your S3 Bucket and file key
bucket = 'kaggle-leash-bio'

parquet_key = 'train.parquet'

parquet_location = f's3://{bucket}/{parquet_key}'

In [6]:
import dask.dataframe as dd

# Define the list of unique proteins
proteins = ['BRD4', 'HSA', 'sEH']

# Read the data from the parquet file into a Dask DataFrame
train_data = dd.read_parquet(parquet_location, engine='pyarrow')

# Define a function to process each protein
def process_protein(protein_name, df):
    # Select rows where 'binds' is 1
    ones = df[df['binds'] == 1].compute()
    # Select 3 times as many rows where 'binds' is 0
    zeros = df[df['binds'] == 0].sample(frac=3 * len(ones) / len(df[df['binds'] == 0]), random_state=42).compute()
    # Concatenate the two subsets
    combined = dd.concat([ones, zeros])
    # Select only the desired columns
    result = combined[['id', 'molecule_smiles', 'binds']]
    return result

# Process each unique protein and store the results
dfs = {}
for protein in proteins:
    df_protein = train_data[train_data['protein_name'] == protein]
    dfs[protein] = process_protein(protein, df_protein)

# Example: Access the DataFrame for a specific protein
df_BRD4 = dfs['BRD4']

# Save the results to disk if needed
df_BRD4.to_parquet(f'df_BRD4.parquet', engine='pyarrow')


/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [8]:
# Example: Access the DataFrame for a specific protein
df_HSA = dfs['HSA']

# Save the results to disk if needed
df_HSA.to_parquet(f'df_HSA.parquet', engine='pyarrow')


In [9]:
# Example: Access the DataFrame for a specific protein
df_sEH = dfs['sEH']

# Save the results to disk if needed
df_sEH.to_parquet(f'df_sEH.parquet', engine='pyarrow')

In [15]:
len(df_BRD4)

1827855

In [2]:

# Open the Parquet file
parquet_file = pq.ParquetFile(test_parquet_location)

NameError: name 'pq' is not defined